#### In this notebook, we convert an audio file of an English speaker to text using a Speech to Text API. Then we translate the English version to a Simplified chinese and spanish version using a Language Translator API.

In [21]:
!pip install ibm_watson wget

In [32]:
# First we import SpeechToTextV1 from ibm_watson.
from ibm_watson import SpeechToTextV1 
import json
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [33]:
# we store the information in the variable URL
url_s2t = "https://api.eu-gb.speech-to-text.watson.cloud.ibm.com/instances/e6f80593-5f3a-4769-9600-93c9b4ea83a3"

In [34]:
iam_apikey_s2t = "Kg5d47HVc67Agp7cZp4gS5KSPpPueuEQmedjXkon5oI9"

In [35]:
# create a Speech To Text Adapter object the parameters are the endpoint and API key
authenticator = IAMAuthenticator(iam_apikey_s2t)
s2t = SpeechToTextV1(authenticator=authenticator)
s2t.set_service_url(url_s2t)
s2t

In [36]:
# download the audio file that we will use to convert into text.
!wget -O PolynomialRegressionandPipelines.mp3  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/PY0101EN/labs/PolynomialRegressionandPipelines.mp3


--2021-01-11 07:10:03--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/PY0101EN/labs/PolynomialRegressionandPipelines.mp3
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4234179 (4.0M) [audio/mpeg]
Saving to: ‘PolynomialRegressionandPipelines.mp3’

PolynomialRegressio 100%[===================>]   4.04M  --.-KB/s    in 0.09s   

2021-01-11 07:10:03 (43.0 MB/s) - ‘PolynomialRegressionandPipelines.mp3’ saved [4234179/4234179]



In [37]:
# path of the wav file we would like to convert to text
filename='PolynomialRegressionandPipelines.mp3'

In [38]:
# We create the file object wav with the wav file using open ; 
we set the mode to "rb" , this is similar to read mode, but it ensures the file is in binary mode.We use the method recognize to return the recognized text. The parameter audio is the file object wav, the parameter content_type is the format of the audio file.
with open(filename, mode="rb")  as wav:
  response = s2t.recognize(audio=wav, content_type='audio/mp3')

In [39]:
response.result

{'result_index': 0,
 'results': [{'final': True,
   'alternatives': [{'transcript': 'in this video we will cover polynomial regression and pipelines ',
     'confidence': 0.94}]},
  {'final': True,
   'alternatives': [{'transcript': "what do we do when a linear model is not the best fit for our data let's look into another type of regression model the polynomial regression we transform our data into a polynomial then use linear regression to fit the parameters that we will discuss pipelines pipelines are way to simplify your code ",
     'confidence': 0.9}]},
  {'final': True,
   'alternatives': [{'transcript': "polynomial regression is a special case of the general linear regression this method is beneficial for describing curvilinear relationships what is a curvilinear relationship it's what you get by squaring or setting higher order terms of the predictor variables in the model transforming the data the model can be quadratic which means the predictor variable in the model is squar

In [40]:
from pandas.io.json import json_normalize

json_normalize(response.result['results'],"alternatives")

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,transcript,confidence
0,in this video we will cover polynomial regress...,0.94
1,what do we do when a linear model is not the b...,0.90
2,polynomial regression is a special case of the...,0.95
3,the model can be cubic which means the predict...,0.95
4,there also exists higher order polynomial regr...,0.91
5,let's look at an example from our data we gene...,0.89
6,in python we do this by using the poly fit fun...,0.92
7,negative one point five five seven X. one cute...,0.90
8,consider the feature shown here applying the m...,0.90
9,pipeline sequentially perform a series of tran...,0.89


In [41]:
response

In [42]:
# obtain the recognized text and assign it to the variable recognized_text
recognized_text=response.result['results'][0]["alternatives"][0]["transcript"]
type(recognized_text)

str

In [44]:
# import LanguageTranslatorV3 from ibm_watson
from ibm_watson import LanguageTranslatorV3

In [46]:
# store the information in the variable URL.
url_lt='https://api.eu-gb.language-translator.watson.cloud.ibm.com/instances/90103e7d-e162-4803-86b3-fdd1f28104de'

In [47]:
# API key
apikey_lt='oajrwc99ce-9iiec6UZNRSIMtZoWyvXRYdJoExj1B4Hm'

In [48]:
# API requests require a version parameter that takes a date in the format version=YYYY-MM-DD.
version_lt='2018-05-01'

In [49]:
# create a Language Translator object language_translator
authenticator = IAMAuthenticator(apikey_lt)
language_translator = LanguageTranslatorV3(version=version_lt,authenticator=authenticator)
language_translator.set_service_url(url_lt)
language_translator

In [50]:
# To get a Lists the languages that the service can identify
from pandas.io.json import json_normalize

json_normalize(language_translator.list_identifiable_languages().get_result(), "languages")

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,language,name
0,af,Afrikaans
1,ar,Arabic
2,az,Azerbaijani
3,ba,Bashkir
4,be,Belarusian
...,...,...
71,uk,Ukrainian
72,ur,Urdu
73,vi,Vietnamese
74,zh,Simplified Chinese


In [60]:
# To translate set it to 'en-zh' or English to simplified chinese
translation_response = language_translator.translate(\
    text=recognized_text, model_id='en-zh')
translation_response

In [61]:
# result
translation=translation_response.get_result()
translation

{'translations': [{'translation': '在这个视频中我们将覆盖多项式回归和管道 '}],
 'word_count': 10,
 'character_count': 64}

In [62]:
# To translate set it to 'en-es' or English to spanish
translation_response = language_translator.translate(\
    text=recognized_text, model_id='en-es')
translation_response

In [63]:
# result
translation=translation_response.get_result()
translation

{'translations': [{'translation': 'en este vídeo cubriremos la regresión polinómica y las tuberías '}],
 'word_count': 10,
 'character_count': 64}

In [70]:
# actual translation as a string as follows:
spanish_translation =translation['translations'][0]['translation']
spanish_translation 

'en este vídeo cubriremos la regresión polinómica y las tuberías '

In [71]:
# we can translate back to English
translation_new = language_translator.translate(text=spanish_translation ,model_id='es-en').get_result()

In [72]:
# obtain the actual translation as a string as follows:
translation_eng=translation_new['translations'][0]['translation']
translation_eng

'in this video we will cover the polynomial regression and the pipes '

In [81]:
# We can convert it to Russian as well:
Russian_translation=language_translator.translate(
  text=translation_eng , model_id='en-ru').get_result()

In [82]:
Russian_translation['translations'][0]['translation']

'в этом видео мы будем покрывать полиномиальную регрессию и трубы '

In [85]:
# We can convert it to arabic as well:
Arabic_translation=language_translator.translate(
  text=translation_eng , model_id='en-ar').get_result()

In [86]:
Arabic_translation['translations'][0]['translation']

'في هذا الفيديو سنغطي الانحسار المتعدد للبدو والمواسير '